In [4]:
!pip install -qU langchain-community langchain-openai rank_bm25 tiktoken faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = input()

In [6]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from langchain.vectorstores import Chroma, FAISS

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [8]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("cryptexcode/mpst-movie-plot-synopses-with-tags")

print("Path to dataset files:", path)
# Assuming the dataset is a CSV file named 'mpst_full_data.csv'
try:
    df = pd.read_csv(path + '/mpst_full_data.csv')
    print(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {path + '/mpst_full_data.csv'}. Check the path or filename.")
except Exception as e:
    print(f"An error occurred: {e}")

Path to dataset files: /root/.cache/kagglehub/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags/versions/1
     imdb_id                                          title  \
0  tt0057603                        I tre volti della paura   
1  tt1733125  Dungeons & Dragons: The Book of Vile Darkness   
2  tt0033045                     The Shop Around the Corner   
3  tt0113862                             Mr. Holland's Opus   
4  tt0086250                                       Scarface   

                                       plot_synopsis  \
0  Note: this synopsis is for the orginal Italian...   
1  Two thousand years ago, Nhagruul the Foul, a s...   
2  Matuschek's, a gift store in Budapest, is the ...   
3  Glenn Holland, not a morning person by anyone'...   
4  In May 1980, a Cuban man named Tony Montana (A...   

                                                tags  split synopsis_source  
0          cult, horror, gothic, murder, atmospheric  train            imdb  
1              

In [9]:
doc_list = []
for index, row in df.iterrows():
    doc_list.append(f"{row['title']}: {row['plot_synopsis']}")

In [10]:
# intialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

In [11]:
bm25_retriever.invoke("Titanic")

[Document(metadata={}, page_content='Keeping Up with the Steins: Benjamin Fiedler (Daryl Sabara) is the 13-year-old son of Jewish couple Adam and Joanne Fiedler (Jeremy Piven and Jami Gertz). After attending the elaborate bar mitzvah party for the son of Arnie Stein (Larry Miller) - which was done on a cruise ship, with a Titanic theme - Benjamin\'s parents decide to go all out for his bar mitzvah. The plan is to rent Dodger Stadium for the bar mitzvah party, complete with movie stars and everything. Adam even kombooks Neil Diamond to sing the National Anthem. However, Benjamin does not want to go through with it, as he does not even understand the words of the haftorah he has to read as part of his bar mitzvah rite. To try to stall the planning, he secretly invites his grandfather Irwin (Garry Marshall), who is now living on an Indian reservation with a New Age woman named Sacred Feather (Daryl Hannah). When Benjamin\'s grandfather arrives, it puts a kink in the planning - as Irwin ha

In [12]:
bm25_retriever.dict

<bound method BaseModel.dict of BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7bff97a97910>, k=2)>

### Embeddings - Dense retrievers FAISS

In [13]:
faiss_vectorstore = FAISS.from_texts(doc_list, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

In [14]:
faiss_retriever.invoke("Dragon")

[Document(id='3ed0c4b5-6811-4245-ba2f-92b9fa654219', metadata={}, page_content='DragonHeart: Dragonheart begins with Bowen (Dennis Quaid), a Knight of the "Old Code" of honor, valor and chivalry, tutoring his young protege, Einon (Lee Oakes) in both swordplay and the values of a knight. Einon is the king\'s son and heir to the throne. The lesson, conducted amongst the ruins of an old Roman fortress, is interrupted by Brok (Brian Thompson), leader of the armies of King Freyne (Peter Hric). Brok tells Einon of a peasant rebellion, and King Freyne wants Einon to witness it being crushed. Einon rushes to the village where the fighting is taking place, forcing Bowen to follow to protect him. Einon thinks Bowen\'s blade will be the best on the battlefield, but Bowen warns Einon to learn the difference between battle and butchery, and only by doing so will he be a greater ruler than his father. Einon promises that he will be a greater ruler.(During the battle, a caption says the movie takes p

In [15]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5])

In [16]:
docs = ensemble_retriever.invoke("Titanic")
docs

[Document(metadata={}, page_content='Keeping Up with the Steins: Benjamin Fiedler (Daryl Sabara) is the 13-year-old son of Jewish couple Adam and Joanne Fiedler (Jeremy Piven and Jami Gertz). After attending the elaborate bar mitzvah party for the son of Arnie Stein (Larry Miller) - which was done on a cruise ship, with a Titanic theme - Benjamin\'s parents decide to go all out for his bar mitzvah. The plan is to rent Dodger Stadium for the bar mitzvah party, complete with movie stars and everything. Adam even kombooks Neil Diamond to sing the National Anthem. However, Benjamin does not want to go through with it, as he does not even understand the words of the haftorah he has to read as part of his bar mitzvah rite. To try to stall the planning, he secretly invites his grandfather Irwin (Garry Marshall), who is now living on an Indian reservation with a New Age woman named Sacred Feather (Daryl Hannah). When Benjamin\'s grandfather arrives, it puts a kink in the planning - as Irwin ha